## Chat Models - <a href='https://python.langchain.com/docs/modules/data_connection/document_loaders/'>Document Loaders</a> and Text Splitting


In [ ]:
%pip install langchain langchain_openai beautifulsoup4 langchain-community --upgrade

In [4]:
from bs4 import BeautifulSoup
from langchain_community.document_loaders import TextLoader
import requests

# Get this file and save it locally:
url = "https://github.com/hammer-mt/thumb/blob/master/README.md"

# Save it locally:
r = requests.get(url)

# Extract the text from the HTML:
soup = BeautifulSoup(r.text, 'html.parser')
text = soup.get_text()

with open("README.md", "w") as f:
    f.write(text)

loader = TextLoader('README.md')
docs = loader.load()

In [5]:
from langchain_core.documents import Document
[ Document(page_content='test', metadata={'test': 'test'}) ]

[Document(metadata={'test': 'test'}, page_content='test')]

In [6]:
# Split the text into sentences:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 300,
    chunk_overlap  = 50,
    length_function = len,
    is_separator_regex = False,
)

final_docs = text_splitter.split_documents(loader.load())

In [7]:
len(final_docs)

79

In [8]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI as genai_chat
from langchain.chains.summarize import load_summarize_chain
load_dotenv()
model_name = "models/gemini-2.0-flash-exp"

chat = genai_chat(model=model_name)

In [13]:
chain = load_summarize_chain(llm=chat, chain_type="map_reduce")
result = chain.invoke({
    "input_documents": final_docs[:10],
})

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 44
}
].
